In [3]:
pip install Langchain


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------  2.1/2.1 MB 13.0 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 11.9 MB/s eta 0:00:00


In [81]:
import langchain

In [82]:
pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install langchain-groq

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [83]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [84]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [85]:
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")



In [86]:
print(os.getenv("GROQ_API_KEY"))
print(GROQ_API_KEY)

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
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


### Example 1: Simple LLM call with Streaming

In [7]:
# Initialize the model:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

from langchain_groq import ChatGroq


In [10]:
model=init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001230548EC30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001230548E120>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [30]:
## Messages:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What are top 2 benefits of LLM?"),
]

In [15]:
## Invoke the model:
response = model.invoke(messages)
print(response.content)

APIConnectionError: Connection error.

In [58]:
from groq import Groq
import httpx

# Disable SSL verification (for debugging only)
http_client = httpx.Client(verify=False)

# Initialize Groq client
client = Groq(api_key=GROQ_API_KEY, http_client=http_client)

# Define messages (similar to LangChain format)
messages1 = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What are top 2 benefits of LLM?"}
]

model1 = "llama-3.1-8b-instant"

# Invoke the model
response = client.chat.completions.create(
    model=model1,
    messages=messages1
)

# Print the response
print(response.choices[0].message.content)


LLM stands for Large Language Model. The top 2 benefits of LLMs are:

1. **Improved Text Generation and Summarization**: Large Language Models can understand and generate human-like text based on the input they receive. They can summarize long pieces of text, create content, and even assist in writing tasks like article writing, email writing, and more.

2. **Enhanced Query Answering and Conversational Capabilities**: LLMs can process and understand natural language queries, which enables them to respond with accurate and relevant information. They can engage in conversation, answer questions, and even learn from user interactions, making them ideal for applications like virtual assistants, chatbots, and customer service platforms.


In [48]:
response = client.chat.completions.create(
    model=model1,
    messages=[{"role": "user", "content": "what is capital of Karnataka?"}] 
)

print(response.choices[0].message.content)

The capital of Karnataka is Bengaluru (also known as Bangalore).


In [ ]:
# Using Stream 
response1 = client.chat.completions.create(
    model=model1,
    messages=messages1,
    stream=True  
)


for chunk in response1:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True
        )



LLM stands for Large Language Model. The top 2 benefits of LLMs include:

1. **Text Generation and Summarization**: LLMs can generate human-like text, summaries, and responses to a wide range of questions or topics. They can assist in tasks such as content creation, rewriting articles, and summarizing long documents into concise versions. These capabilities make LLMs incredibly useful for writing, research, and communication tasks.

2. **Language Understanding and Analysis**: LLMs can comprehend and analyze language patterns, sentiment, and context, enabling them to analyze vast amounts of text data. This ability helps with tasks such as natural language processing (NLP), machine translation, and text classification, making LLMs valuable tools for applications like chatbots, language translation software, and sentiment analysis tools.

These benefits have far-reaching implications across various industries, including education, healthcare, and customer service.

## Daynamic Prompt Template

In [63]:
from langchain_core.prompts import ChatPromptTemplate

translation_template = ChatPromptTemplate.from_messages([(
    "system", "You are a helpful assistant that translates the following {text} from {source_language} to {target_language}. Maintain the tone and style"),
    ("human", "{text}")

])

## using the template
prompt = translation_template.invoke({
    "source_language": "English",
    "target_language": "French",
    "text": "Hello, how are you?"   
})


In [65]:
prompt
#translated_response = model.invoke(prompt)
#print(translated_response.content)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that translates the following Hello, how are you? from English to French. Maintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

In [ ]:
import groq
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Configure Groq API
groq.api_key = "your-groq-api-key"  # or set via environment variable
groq.api_base = "https://api.groq.com/openai/v1"
groq.verify_ssl = False  # Disable SSL verification

# Define a function that uses Groq SDK directly
def call_groq_model(prompt_messages):
    response = groq.ChatCompletion.create(
        model="llama-3.1-8b-instant",
        messages=prompt_messages
    )
    return response.choices[0].message.content

# Define the chain
def create_story_chain():
    story_template = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short and engaging story based on theme."),
        ("user", "Theme: {theme} \n Main character: {character} \n setting: {setting}")
    ])

    analysis_template = ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the story and provide feedback on it"),
        ("user", "{story}")
    ])

    # Use RunnableLambda to call Groq directly
    story_chain = story_template | RunnableLambda(
        lambda inputs: call_groq_model([
            {"role": "system", "content": "You are a creative storyteller. Write a short and engaging story based on theme."},
            {"role": "user", "content": f"Theme: {inputs['theme']}\nMain character: {inputs['character']}\nsetting: {inputs['setting']}"}
        ])
    ) | StrOutputParser()

    def analyze_story(story_text):
        return {"story": story_text}

    analysis_chain = story_chain | RunnableLambda(analyze_story) | RunnableLambda(
        lambda inputs: call_groq_model([
            {"role": "system", "content": "You are a literary critic. Analyze the story and provide feedback on it"},
            {"role": "user", "content": inputs["story"]}
        ])
    ) | StrOutputParser()

    return analysis_chain

# Run the chain
chain = create_story_chain()

result = chain.invoke({
    "theme": "Friendship",
    "character": "Alice",
    "setting": "A small village"
})

print("Story and Analysis:")
print(result)


"Bonjour, comment allez-vous?"


## Building your First Chain

In [72]:
from langchain_core.output_parsers import StrOutputParser

def create_story_chain():
    ## Templeate for story generation
    story_template = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short and engaging story based on theme."),
        ("user", "Theme: {theme} \n Main character: {character} \n setting: {setting}")
    ])

    analysis_template = ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the story and provide feedback on it"),
        ("user", "{story}")
    ])

    story_chain=(
        #story_template | model1 | analysis_template | model1
        story_template | model | StrOutputParser()
    )

    analysis_chain = (
        {"story": story_chain} | analysis_template | model | StrOutputParser()
    )

    return analysis_chain

    

In [73]:
chain=create_story_chain()
chain

{
  story: ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short and engaging story based on theme.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme} \n Main character: {character} \n setting: {setting}'), additional_kwargs={})])
         | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001230548EC30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001230548E120>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
         | StrOutputParser()
}
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variables={},

In [75]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def create_story_chain():
    ## Templeate for story generation
    story_template = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short and engaging story based on theme."),
        ("user", "Theme: {theme} \n Main character: {character} \n setting: {setting}")
    ])

    analysis_template = ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the story and provide feedback on it"),
        ("user", "{story}")
    ])

    story_chain=(
        #story_template | model1 | analysis_template | model1
        story_template | model | StrOutputParser()
    )

    #create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story":story_text}

    analysis_chain = (
        story_chain | RunnableLambda(analyze_story) | analysis_template | model | StrOutputParser()
    )

    return analysis_chain

    

In [76]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short and engaging story based on theme.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme} \n Main character: {character} \n setting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001230548EC30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001230548E120>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variables={}

In [77]:
result = chain.invoke({
    "theme": "Friendship", 
    "character": "Alice",
    "setting": "A small village"
})

print("Story and Analysis: ")
print(result)

APIConnectionError: Connection error.

In [96]:
import httpx
from groq import Groq
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create a custom HTTPX client with SSL verification disabled
transport = httpx.HTTPTransport(verify=False)
httpx_client = httpx.Client(transport=transport, base_url="https://api.groq.com/openai/v1")

# Initialize Groq client and patch it to use the custom HTTPX client
groq_client = Groq(api_key=GROQ_API_KEY)
groq_client._client = httpx_client

# Function to call Groq using LangChain-compatible input
def call_groq(prompt_value):
    # Map LangChain message types to Groq-compatible roles
    role_map = {
        "system": "system",
        "human": "user",
        "ai": "assistant"
    }

    messages = [
        {"role": role_map.get(m.type, "user"), "content": m.content}
        for m in prompt_value.to_messages()
    ]

    response = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=messages,
        temperature=0.7,
        max_tokens=1024,
        top_p=1,
        stream=False
    )
    return response.choices[0].message.content




# Define the chain
def create_story_chain():
    story_template = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short and engaging story based on theme."),
        ("user", "Theme: {theme} \n Main character: {character} \n setting: {setting}")
    ])

    analysis_template = ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the story and provide feedback on it"),
        ("user", "{story}")
    ])

    story_chain = story_template | RunnableLambda(call_groq) | StrOutputParser()

    def analyze_story(story_text):
        return {"story": story_text}

    analysis_chain = story_chain | RunnableLambda(analyze_story) | analysis_template | RunnableLambda(call_groq) | StrOutputParser()

    return analysis_chain

# Run the chain
chain = create_story_chain()

result = chain.invoke({
    "theme": "Friendship",
    "character": "Alice",
    "setting": "A small village"
})

print("Story and Analysis:")
print(result)


Story and Analysis:
**Analysis and Feedback**

"The Starlight Serenade" is a heartwarming tale that explores the transformative power of friendship and community. On the surface, it appears to be a simple, feel-good story, but upon closer inspection, it reveals a rich tapestry of themes, characters, and symbolism.

**Strengths:**

1. **Immersive Setting**: The author skillfully transports readers to the quaint village of Willowdale, evoking a sense of wonder and enchantment. The descriptions of the rolling hills, sparkling lake, and blooming wildflowers effectively capture the essence of a picturesque summer evening.
2. **Vibrant Characters**: Alice and Emilia are well-crafted characters, each with their unique strengths and weaknesses. Their friendship is authentic and endearing, making their interactions a joy to read.
3. **Poetic Language**: The author's use of language is lyrical and evocative, with phrases like "the whispers of the wind," "the lapping of the waves," and "the rustl

In [91]:
import httpx
from groq import Groq

# Create a custom HTTPX client with SSL verification disabled
transport = httpx.HTTPTransport(verify=False)
client = httpx.Client(transport=transport, base_url="https://api.groq.com/openai/v1")

# Patch Groq SDK to use this client
groq_client = Groq(api_key=GROQ_API_KEY)
groq_client._client = client  # override internal client

# Now use it
response = groq_client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a story about friendship in a small village."}
    ]
)

print(response.choices[0].message.content)



In the heart of a small village, nestled between rolling hills and lush green forests, lived two friends, Akira and Taro. They had grown up together, exploring the winding streets and secret meadows that only the villagers knew existed.

Akira was the village's skilled baker, famous for his delectable pastries and bread that always seemed to appear at just the right moment. He took pride in his work, and his smile could light up the entire village whenever he stepped out of his cozy bakery, with its mouthwatering aromas trailing behind him.

Taro, on the other hand, was a kind-hearted farmer who worked tirelessly to maintain the village's lush gardens and greenhouses. His gentle touch coaxed even the most reluctant plants to bloom, and his love for nature was contagious. Whenever he'd stroll through the village, people would stop to chat with him and admire the beauty he cultivated.

As the seasons changed, the friends grew closer. Akira would often sneak into Taro's greenhouses to pic